This notebook is dedicated to the cleaning and preprocessing of text extracted from the document in sentence form. It involves refining the text by removing or cleaning elements such as URLs, footnotes, and titles of figures & tables. Users can directly utilize this notebook following the 01_input.ipynb if they wish to include all sections of the document in their analysis. However, if users wish to exclude certain sections from their analysis, they should first utilize the 00_optional_customize_scope_by_toc.ipynb notebook to select these sections. Following this, they need to update the drop_headings variable in 01_input.ipynb with the sections they have chosen to omit and run the file again. Once this update is made, users can then run this notebook, which will process the text reflecting the specified omissions. An updated table of contents will be printed to verify the sections included in the analysis.

Please ensure to paste the input_path, which is the location of the output of the 01_input.ipynb file, at the start of this notebook. This step is the only requirement to load all necessary information for the execution of the code.

At the end of this notebook, the processed sentences extracted from paragraphs and tables are displayed in the variables text_body and text_table, respectively.

Recommended Google Colab Runtime Type: CPU (default).

In [ ]:
# Specify the directory path where the output of the input file 01_input.ipynb was saved
input_path = "/content/drive/My Drive/ImpactDataMining/Hurricane_Ian/01_Input"

All the below sections automatically retrieve data from the 01_input.ipynb file, as well as results from previous notebooks in this series. The code is designed to run using this information, so no further edits are required beyond this point.

In [ ]:
!pip install python-docx
!pip install anytree

import docx
import nltk; nltk.download(['punkt', 'averaged_perceptron_tagger'])
import os
import json
import re
import numpy as np
import pandas as pd

from google.colab import drive
from anytree import Node, RenderTree, search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 773.3 kB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
import time
start_time = time.time()

In [ ]:
def current_path():
  print("Current working directory")
  print(os.getcwd())
  print()

current_path()
drive.mount('/content/drive')
os.chdir(input_path)
current_path()

Current working directory
/content

Mounted at /content/drive
Current working directory
/content/drive/My Drive/ImpactDataMining/Hurricane_Ian/01_Input



In [ ]:
with open('0_input.json', 'r') as file:
    data = json.load(file)
    document_path = data['document_path']
    output_path = data['output_path']
    document_name = data['document_name']
    drop_headings = data['drop_headings']

In [ ]:
drive.mount('/content/drive')
os.chdir(document_path)
current_path()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory
/content/drive/My Drive/ImpactDataMining/Hurricane_Ian/01_Input



In [ ]:
doc = docx.Document(document_name)

names = []
for para in doc.paragraphs:
  names.append(para.style.name)

text = []
for para in doc.paragraphs:
  text.append(para.text)

text_table = []
for table in doc.tables:
  table_data = []
  for row in table.rows:
    row_data = []
    for cell in row.cells:
        row_data.append(cell.text)
    table_data.append(row_data)
  text_table.append(table_data)

In [ ]:
drive.mount('/content/drive')
os.chdir(output_path)
current_path()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory
/content/drive/My Drive/ImpactDataMining/Hurricane_Ian/03_Output



In [ ]:
def remove_headings(root, heading):
  node_to_remove = search.find_by_attr(root, name="name", value=heading)

  # List to store all nodes that will be removed
  removed_nodes = []

  if node_to_remove:
      # Get all descendants of the node
      removed_nodes.append(node_to_remove)
      removed_nodes.extend(node_to_remove.descendants)

      # Detach the node, which also removes all of its descendants
      node_to_remove.parent = None
      remove_nodes = [n.name for n in removed_nodes]
  return [n.name for n in removed_nodes]

def remove_headings_idx(removed_nodes, headings, headings_idx):
  idx_omit = []
  for i, n in enumerate(removed_nodes):
    idx_omit.extend([headings_idx[i1] for i1, n1 in enumerate(headings) if n1 == n])
    if i == len(removed_nodes)-1:
      if n == headings[-1]:
        idx_omit.append(len(text))
      else:
        idx_omit.extend([headings_idx[i1+1] for i1, n1 in enumerate(headings) if n1 == n])
  return idx_omit

In [ ]:
if len(drop_headings) > 0:
  headings_tree = [(names[i], text[i]) for i, n in enumerate(names) if n.startswith('Heading')]
  headings = [text[i] for i, n in enumerate(names) if n.startswith('Heading')]
  headings_idx = [i for i, n in enumerate(names) if n.startswith('Heading')]

  # Root node
  root = Node("Document")

  # Keep track of the last node for each heading level
  last_nodes = {0: root}

  for level, title in headings_tree:
    level_num = int(level.split(' ')[-1])

    # Reset the last nodes for levels greater than the current level
    for i in range(level_num + 1, max(last_nodes.keys()) + 1):
      if i in last_nodes:
        del last_nodes[i]

    # Find the correct parent by looking at the last node at the previous level or above
    parent = None
    for i in range(level_num - 1, -1, -1):
      parent = last_nodes.get(i)
      if parent:
        break

    # Create the new node and store it as the last node for its level
    node = Node(title, parent=parent)
    last_nodes[level_num] = node

  drop_idx = []
  for n in drop_headings:
    removed_nodes = remove_headings(root, n)
    idx = remove_headings_idx(removed_nodes, headings, headings_idx)
    if len(idx) > 2:
      idx = [idx[0], idx[-1]]
    drop_idx.append(idx)

  # Print the tree
  print('UPDATED TABLE OF CONTENTS')
  for pre, _, node in RenderTree(root):
    print(f"{pre}{node.name}")

  # Start by reverse sorting the indices so that the order of elements is not disturbed while removing
  for n in drop_idx[::-1]:
    for k in range(len(n)-1):
      start_idx = n[k]
      end_idx = n[k+1]
      del text[start_idx:end_idx]
      del names[start_idx:end_idx]

UPDATED TABLE OF CONTENTS
Document
└── EXECUTIVE SUMMARY
    ├── Introduction
    │   ├── Loss of Life and Injuries
    │   ├── Official Response
    │   └── Report Scope
    ├── Hazard Characteristics
    │   ├── Meteorological Background
    │   ├── Wind Field
    │   ├── Storm Surge and Coastal Flooding
    │   ├── Rainfall and Inland Flooding
    │   ├── Tornadoes
    │   └── Comparative Case: Hurricane Charley (2004)
    ├── Local Codes and Construction Practices
    ├── Building Performance
    ├── Infrastructure Performance
    │   ├── Power Outages & Restoration
    │   │   └── Power Case Study: Solar Panels
    │   ├── Transportation Disruptions & Restoration
    │   │   └── Transportation Case Study: Sanibel Causeway
    │   └── Telecommunications
    └── Coastal Protective Systems Performance


In [ ]:
text_normal = [text[i] for i, n in enumerate(names) if n.startswith("Normal")]
text_normal = [n for n in text_normal if n != '' and n != '\t']

# footnotes handling
fnote_char_idx = []; fnote_num_idx = [];
for i, n in enumerate(text_normal):
  if re.findall('^[a-z]\s*[A-Z0-9\u201c\u2018|https]', n):
    fnote_char_idx.append(i)
  if re.findall('^\d+\s*[A-Z0-9\u201c\u2018|https]', n):
    fnote_num_idx.append(i)
fnote_num = [int(re.match(r'^\d+',text_normal[i]).group()) for i in fnote_num_idx]
fnote_char = [re.match(r'^[a-z]',text_normal[i]).group() for i in fnote_char_idx]

result = []; result_idx_num = [];
for num, idx in zip(fnote_num, fnote_num_idx):
    if not result:
        if num == 1:
            result.append(num)
            result_idx_num.append(idx)
    elif num == result[-1] + 1:
        result.append(num)
        result_idx_num.append(idx)
    else:
        continue
result = []; result_idx_char = [];
for num, idx in zip(fnote_char, fnote_char_idx):
    if not result:
        if num == "a":
            result.append("a")
            result_idx_char.append(idx)
    elif num == chr(ord(result[-1]) + 1):
        result.append(num)
        result_idx_char.append(idx)
    else:
        continue

fnote_idx = list(set(result_idx_num+result_idx_char))
text_fnote = [text_normal[i] for i in fnote_idx]
fnote_idx = [text.index(n) for n in text_fnote]
text = [text[i] for i in range(len(text)) if i not in fnote_idx]
names = [names[i] for i in range(len(names)) if i not in fnote_idx]

In [ ]:
# https handling
for i, n in enumerate(text):
  if re.findall('http[s]?://\S+', n):
    text[i] = re.sub(r'http[s]?://\S+', '', n)

In [ ]:
# Table, Figure titles handling
idx_heading = [i for i, n in enumerate(names) if n.startswith("Heading")]
b = [n for n in range(len(text)) if n not in idx_heading]
text_body = [text[i] for i in b]
text_body = [n for n in text_body if n != ""]

idx_fig_table = [];
for i, n in enumerate(text_body):
  pattern = r'^(?:Table|Figure)\s(?:\d|[A-Z])+(?:\.\d+)*(?:\.|:)\s'
  match = re.search(pattern,n)
  if match:
    if i != 0:
      match1 = re.search(r'\(a\).*\t', text_body[i-1])
      if match1:
        idx_fig_table.extend(list(range(i-1,i+1)))
      else:
        idx_fig_table.append(i)
    else:
      idx_fig_table.append(i)

idx = [text_body.index(n) for n in text_body if text_body.index(n) not in idx_fig_table]
text_body = [text_body[i] for i in idx]
for i, n in enumerate(text_body):
  if n.strip().endswith(":"):
    text_body[i] = re.sub(":", ".", n)

# Headings handling
idx_1 = []; idx_1_n = [];
for n in text_body:
  if ":" in n and n.strip().endswith('.') is False:
    t = n[:n.index(":")]
    text_tokenized = nltk.word_tokenize(t)
    text_tokenized = [w for w in text_tokenized if re.findall(r'\w+', w)]
    text_pos_tagged = nltk.pos_tag(text_tokenized)
    idx_1.append(text_body.index(n))

    for k in text_pos_tagged:
      if not re.match('NN', k[1]):
        idx_1_n.append(text_body.index(n))
        break
idx_1 = [i for i in idx_1 if i not in idx_1_n]
idx = [i for i in range(len(text_body)) if i not in idx_1]
text_body = [text_body[i] for i in idx]

In [ ]:
idx_1 = [i for i, n in enumerate(text_body) if n.strip().endswith(".") is False]
temp = []; temp_last = idx_1[-1];
for i in range(len(idx_1)-1):
    if idx_1[i+1] - idx_1[i] == 1:
        temp.append(idx_1[i+1])

# spelling
for i, para in enumerate(text_body):
  text_body[i] = re.sub(r'e\.g\.\s+', 'e.g.', para)
  text_body[i] = re.sub(r'et al\.\s+', 'et al.', text_body[i])
  text_body[i] = re.sub(r'(?=.)Fig. ', 'Fig.', text_body[i])
  text_body[i] = re.sub(r'No\.\s+', 'No.', text_body[i])
  text_body[i] = re.sub(r'deg\.', 'degree', text_body[i])
  text_body[i] = re.sub(r'\)$', ').', text_body[i])

In [ ]:
idx_1 = [i for i in idx_1 if i not in temp]
idx_2 = []
for i in range(len(idx_1)-1):
  for k in range(idx_1[i],idx_1[i+1]):
    if text_body[k].strip().endswith("."):
      idx_2.append(k)
      break
if len(idx_2) < len(idx_1):
  idx_2.append(temp_last)

for n in range(len(idx_1)):
    text_body[idx_1[n]] = " ".join(text_body[idx_1[n]:idx_2[n]+1])

for n in range(len(idx_1))[::-1]:
    text_body[idx_1[n]+1:idx_2[n]+1] = ""

In [ ]:
def table_format(table):
  for i, line in enumerate(table):
    for i1, col in enumerate(line):
      pattern1 = r'^(?:Table|Figure)\s(?:\d|[A-Z])+(?:\.\d+)*(?:\.|:)\s'
      match1 = re.search(pattern1, col)
      if match1:
        table[i][i1] = ""

      pattern2 = 'http[s]?://\S+'
      match2 = re.search(pattern2, col)
      if match2:
         table[i][i1]= re.sub(r'http[s]?://\S+', '', col)

  for i, line in enumerate(table):
    if all(col == "" for col in line):
      table.pop(i)

  try:
    return table_format1(table)
  except IndexError:
    return table

def table_format1(table):
  ta_idx = []
  for i, line in enumerate(table):
    if line[0] != "":
      ta_idx.append(1)
    else:
      ta_idx.append(0)

  if (ta_idx[0] == 0 and 0 not in ta_idx[1:]) or 0 not in ta_idx:
    return table
  else:
    idx = []
    count = 0; zeros = 0
    for i, item in enumerate(ta_idx):
      if item == 1:
        count = count + 1
      else:
        zeros = zeros + 1
        if count > 0 and (i == len(ta_idx) - 1 or ta_idx[i+1] == 1):
          count = count +  zeros
          idx.append(count)
          count = 0; zeros = 0
    if count > 0:
      idx.append(count)

    table = [list(x) for x in zip(*table)]
    for i, n in enumerate(table):
      result = []
      start = 0
      for i1 in idx:
        end = start + i1
        result.append(" ".join(n[start:end]))
        start = end
      table[i] = result
    return [list(x) for x in zip(*table)]

def is_number(string):
    if string.isnumeric():  # Check if the string represents an integer
        return True
    else:
        try:
            float(string)  # Try to convert the string to a float
            return True
        except ValueError:  # If the conversion fails, it's not a number
            return False

def combine_table(tables):
  for t1, t2 in zip(tables[:-1], tables[1:]):
    if len(t1[-1]) == len(t2[0]):
      for i, (c1, c2) in enumerate(zip(t1[-1][1:], t2[0][1:])): #starting from 2nd column
        if c1.endswith(".") is False and is_number(c1) is False:
          t1[-1][i+1] = " ".join([c1, c2])
          t2[0][i+1] = ""

  for i, t in enumerate(tables):
    for i1, l in enumerate(t):
      if all(c == "" for c in l):
        tables[i].pop(i1)
  return tables

def remove_short_lists(lst):
    # If the element is not a list, return it as is.
    if not isinstance(lst, list):
        return lst

    # Process the list elements
    result = []
    for item in lst:
        # Recursively apply the function to sublists
        if isinstance(item, list):
            processed_item = remove_short_lists(item)
            # Add the item to the result if it's a non-empty list with at least 2 elements
            if isinstance(processed_item, list) and len(processed_item) > 0:
                result.append(processed_item)
        else:
            # Directly add non-list items
            result.append(item)
    return result

In [ ]:
for i, t in enumerate(text_table):
  text_table[i] = table_format(t)

text_table = [t for t in text_table if t]
for t in text_table:
  for i, c  in enumerate(t):
    t[i] = [r for r in c if len(r) > 5]
text_table = remove_short_lists(text_table)

text_table = combine_table(text_table)
text_table = list(dict.fromkeys([c for t in text_table for r in t for c in r]))

In [ ]:
def identify_sentence(sent):
  for i, item in enumerate(sent):
    match_VB = re.search('VB.*', item[1])
    if match_VB:
      return sent
    else:
      if i == len(sent) - 1:
        sent = ''
        return sent
      else:
        continue

def identify_sentence2(sent):
  try:
    match_VB = re.search('VB.', sent[0][1])
    if match_VB:
      return identify_sentence(sent[1:])
    else:
      return identify_sentence(sent)
  except IndexError:
    pass

def data_sentences(data):
  n = len(data);
  sent_tokenized = [None]*n; sent_pos_tagged = [None]*n; sent = [None]*n;
  idx = [None]*n; result = [None]*n;
  sentences = [nltk.sent_tokenize(p) for p in data]
  for i, n in enumerate(sentences):
    sentences[i] = [k for k in n if len(k.split())>=4]
  for i, n in enumerate(sentences):
    sent_tokenized[i] =  [nltk.word_tokenize(k) for k in n]
  for i, n in enumerate(sent_tokenized):
    sent_pos_tagged[i] =  [nltk.pos_tag(k) for k in n]
  for i, n in enumerate(sent_pos_tagged):
    sent[i] =  [identify_sentence2(k) for k in n]
  for i, n in enumerate(sent):
    idx[i] = [0 if k == '' else 1 for k in n]
  for i, (a, b) in enumerate(zip(sentences, idx)):
    result[i] = [s for s, d in zip(a, b) if d == 1]
  result = [n for n in result if n != []]
  idx = [n for n in [list(range(len(n))) for n in result]]
  result = [k for n in result for k in n]
  return result, idx

In [ ]:
text_body, idx_body = data_sentences(text_body)
text_table, idx_table = data_sentences(text_table)

for i, n in enumerate(text_table):
      pattern_1 = r'^\n'
      pattern_2 = r'\([a-z]\)\s*'
      pattern_3 = r'\n'

      n = re.sub(pattern_1, '', n)
      n = re.sub(pattern_2, '', n)
      text_table[i] = re.sub(pattern_3, ' - ', n)

In [ ]:
text_body

['Eighteen years after Hurricane Charley made landfall in 2004, Hurricane Ian made landfall in nearly the same location, also as a Category 4 hurricane.',
 'Unlike Hurricane Charley (2004), water more so than wind was the impetus behind the disaster that unfolded.',
 'Despite being a below- design-level wind event, the large windfield drove a powerful storm surge as much as 13 ft high (relative to the NAVD8 vertical datum) in the barrier islands of Sanibel, Ft. Myers Beach, and Bonita Beach.',
 'Flooding was extensive along not only the Florida coast, but also well inland into low-lying areas as far north as Duval County and the storm’s second landfall site in South Carolina.',
 'As such, Hurricane Ian will likely be one of the costliest landfalling hurricanes of all time in the US, claiming over 100 lives.',
 'The impacts from Hurricane Ian were most severe in the barrier islands from the combination of storm surge and high winds, with many buildings completely washed away, and others

In [ ]:
text_table

['Hurricane Ian - Released: November 16, 2022 NHERI DesignSafe Project ID: PRJ- 3709',
 'Academic Organization Focusing on Earthquake Hazard',
 '*prematurely failed; peak effects possibly not captured (FEMA, 2005)',
 'Single-family housing performance was variable, with exposure to surge hazard being a primary driver of structural failures.',
 'Such damage was primarily restricted to the barrier islands (e.g., Sanibel, Fort Myers Beach, Bonita Springs), where many homes were completely washed away and many others lost breakaway walls.',
 'Mobile/manufactured housing and RV parks were the most susceptible to the damage as they lacked the elevation and lateral force resisting systems necessary to resist any significant storm surge-induced loads.',
 'Wind damage was primarily limited to building envelopes in site-built homes, and even that was isolated based on the imagery available at the time of this report.',
 'More severe damage, including structural failures, were observed in some mo

In [ ]:
# Saving results to a JSON file
with open('1_output.json', 'w') as file:
    json.dump({
        'text_body': text_body, 'text_table': text_table,
        'idx_body': idx_body, 'idx_table': idx_table
        }, file)

# Saving results to an excel file
df1 = pd.DataFrame(text_body)
df2 = pd.DataFrame(text_table)

with pd.ExcelWriter('1_output.xlsx', engine='openpyxl') as writer:
    df1.to_excel(writer, sheet_name='Paragraph text', index=False, header=False)
    df2.to_excel(writer, sheet_name='Table text', index=False, header=False)

In [ ]:
end_time = time.time()
execution_time = end_time - start_time

print("Execution time:", execution_time, "seconds")

Execution time: 29.63161587715149 seconds
